[View the runnable example on GitHub](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/pytorch/pytorch_save_and_load_ipex.ipynb)

# Save and Load Optimized IPEX Model

This example illustrates how to save and load a model accelerated by IPEX.

In this example, we use a pretrained ResNet18 model. Then, by calling `InferenceOptimizer.trace(..., use_ipex=True)`, we can obtain a model accelerated by IPEX method. By calling `InferenceOptimizer.save(model=..., path=...)` , we could save the Nano optimized model to a folder. By calling `InferenceOptimizer.load(path=..., model=original_model)`, we could load the IPEX optimized model from a folder.

To conduct IPEX optimization through BigDL-Nano InferenceOptimizer, you need to install BigDL-Nano for PyTorch first. We recommend you to use [Miniconda](https://docs.conda.io/en/latest/miniconda.html) to prepare the environment and install the following packages in a conda environment.

You can create a conda environment by executing:

```
# "nano" is conda environment name, you can use any name you like.
conda create -n nano python=3.7 setuptools=58.0.4
conda activate nano
```
> 📝 **Note**
>
> During your installation, there may be some warnings or errors about version, just ignore them.

In [ ]:
# Necessary packages for inference accelaration using IPEX
!pip install --pre --upgrade bigdl-nano[pytorch]

First, prepare model. We need to load the pretrained ResNet18 model:

In [ ]:
import torch
from torchvision.models import resnet18

model_ft = resnet18(pretrained=True)

## Accelerate Inference Using IPEX

In [ ]:
from bigdl.nano.pytorch import InferenceOptimizer
ipex_model = InferenceOptimizer.trace(model_ft,
                                      use_ipex=True)

## Save Optimized IPEX Model

The saved model files will be saved at "./optimized_model_ipex" directory.

There are 2 files in optimized_model_ipex, users only need to take "ckpt.pth" file for further usage:

* nano_model_meta.yml: meta information of the saved model checkpoint
* ckpt.pth: pytorch state dict checkpoint for general use, describes model structure

In [ ]:
InferenceOptimizer.save(ipex_model, "./optimized_model_ipex")

## Load the Optimized Model

In [ ]:
loaded_model = InferenceOptimizer.load("./optimized_model_ipex", model=model_ft)

> 📝 **Note**
>
> The original model is required when we load the IPEX optimized model. As we only store the `state_dict`, which is simply a python dictionary object that maps each layer to its parameter tensor, when saving the IPEX optimized model.
>
> Note that when the model is optimized by IPEX as well as the JIT accelerator, the original model is not required when loading the optimized model.

## Inference with the Loaded Model

In [ ]:
with InferenceOptimizer.get_context(loaded_model):
    x = torch.rand(2, 3, 224, 224)
    y_hat = loaded_model(x)
    predictions = y_hat.argmax(dim=1)
    print(predictions)

> 📚 **Related Readings**
>
> - [How to install BigDL-Nano](https://bigdl.readthedocs.io/en/latest/doc/Nano/Overview/install.html)